<a href="https://colab.research.google.com/github/peteray-dev/Text-generation-using-gemma/blob/master/LLM_prompt_recovery_using_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generation of datas Using Gemma (Google)

In [1]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


In [2]:
!kaggle competitions download llm-prompt-recovery


  0% 0.00/1.45k [00:00<?, ?B/s]
100% 1.45k/1.45k [00:00<00:00, 2.57MB/s]


In [3]:

!mkdir llm-prompt-recovery

!mv /content/llm-prompt-recovery.zip* /content/llm-prompt-recovery


In [4]:
! unzip "/content/llm-prompt-recovery/llm-prompt-recovery.zip"

Archive:  /content/llm-prompt-recovery/llm-prompt-recovery.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [5]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [6]:
train_df

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [7]:
train_df['rewrite_prompt'][0]

'Convert this into a sea shanty: """The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set."""'

## Generating another text dataset

In [8]:
!kaggle datasets download nelgiriyewithana/emotions

! unzip "emotions.zip"


 51% 8.00M/15.7M [00:00<00:00, 76.7MB/s]
100% 15.7M/15.7M [00:00<00:00, 93.2MB/s]
Archive:  emotions.zip
  inflating: text.csv                


In [9]:
emotion_df = pd.read_csv('/content/text.csv')
# emotion_df = pd.read_csv('/content/llm-prompt-recovery/test.csv')
emotion_df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [10]:
emotion_df.set_index('Unnamed: 0', inplace=True)


In [11]:
emotion_df.head()

,text,label
Unnamed: 0,,
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [12]:
emotion_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416809 entries, 0 to 416808
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    416809 non-null  object
 1   label   416809 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [13]:
original_text = emotion_df['text'][:5]

original_text

Unnamed: 0
0        i just feel really helpless and heavy hearted
1    ive enjoyed being able to slouch about relax a...
2    i gave up my internship with the dmrg and am f...
3                           i dont know i feel so lost
4    i am a kindergarten teacher and i am thoroughl...
Name: text, dtype: object

In [14]:
rewrite_prompts = [
    'Explain this to me like I\'m five.',
    'how to find an internship.',
    'Make this rhyme.',
]

# Let's generate the written the using gemma

In [15]:
!pip install -q -U immutabledict sentencepiece
# cloning gemma model
!git clone https://github.com/google/gemma_pytorch.git

#Moving the model into gemma model file
!mkdir gemma_model
!mv /content/gemma_pytorch/gemma* /content/gemma_model


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.8 MB/s eta 0:00:00
Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 139 (delta 42), reused 40 (delta 24), pack-reused 68
Receiving objects: 100% (139/139), 2.15 MiB | 5.57 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [16]:
import sys
sys.path.append("/content/gemma_model")
from gemma.config import GemmaConfig, get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch


In [17]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/gemma/pyTorch/7b-it-quant")

print("Path to model files:", path)

Attaching model 'google/gemma/pyTorch/7b-it-quant' to your Colab notebook...


Path to model files: /kaggle/input/gemma/pytorch/7b-it-quant/2


In [18]:

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
    print("GPU Device Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

GPU is available
GPU Device Name: Tesla T4


In [19]:
# model loading

weights_dir = '/kaggle/input/gemma/pytorch/7b-it-quant/2'
# variant = '7B-it-quant'
# device_type = 'cuda'

# Load the model
VARIANT = "7b-it-quant"
MACHINE_TYPE = "cuda"
# device = "cuda" if torch.cuda.is_available() else "cpu"

# weights_dir = '/kaggle/input/gemma/pytorch/7b-it-quant/2'

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

# Model Config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = os.path.join(weights_dir, "tokenizer.model")
model_config.quant = "quant" in VARIANT

# Model.
device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
  model.load_weights(ckpt_path)
  model = model.to(device).eval()


In [20]:
# Now we can loop through our input texts, randomly select a rewrite prompt, and see Gemma in action:

import random
random.seed(0)
# This is the prompt format the model expects
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

rewrite_data = []

for original_text in original_text:
    rewrite_prompt = random.choice(rewrite_prompts)
    prompt = f'{rewrite_prompt}\n{original_text}'
    rewritten_text = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=100,
    )
    rewrite_data.append({
        'original_text': original_text,
        'rewrite_prompt': rewrite_prompt,
        'rewritten_text': rewritten_text,
    })

In [21]:
rewrite_data_df = pd.DataFrame(rewrite_data)
rewrite_data_df[:1].values

array([['i just feel really helpless and heavy hearted',
        'how to find an internship.',
        '**Finding an Internship: A Guide to Overcoming Helplessness**\n\n**1. Identify Your Interests and Skills:**\n- Reflect on your interests, skills, and goals to determine industries or fields that align with your strengths and aspirations.\n- Consider your areas of study, experience, and any relevant skills you possess.\n\n**2. Research Companies and Programs:**\n- Explore companies and internship programs that match your interests and criteria.\n- Research their websites, LinkedIn profiles, and industry reputation']],
      dtype=object)

In [30]:
rewrite_data_df[:10].values

array([['i just feel really helpless and heavy hearted',
        'how to find an internship.',
        '**Finding an Internship: A Guide to Overcoming Helplessness**\n\n**1. Identify Your Interests and Skills:**\n- Reflect on your interests, skills, and goals to determine industries or fields that align with your strengths and aspirations.\n- Consider your areas of study, experience, and any relevant skills you possess.\n\n**2. Research Companies and Programs:**\n- Explore companies and internship programs that match your interests and criteria.\n- Research their websites, LinkedIn profiles, and industry reputation'],
       ['ive enjoyed being able to slouch about relax and unwind and frankly needed it after those last few weeks around the end of uni and the expo i have lately started to find myself feeling a bit listless which is never really a good thing',
        'how to find an internship.',
        '**How to Find an Internship**\n\n**1. Identify Your Interests and Skills:**\n- Re

In [22]:
rewrite_data_df

,original_text,rewrite_prompt,rewritten_text
0,i just feel really helpless and heavy hearted,how to find an internship.,**Finding an Internship: A Guide to Overcoming...
1,ive enjoyed being able to slouch about relax a...,how to find an internship.,**How to Find an Internship**\n\n**1. Identify...
2,i gave up my internship with the dmrg and am f...,Explain this to me like I'm five.,It's okay to feel sad when you give up an inte...
3,i dont know i feel so lost,how to find an internship.,**How to Find an Internship**\n\n**1. Determin...
4,i am a kindergarten teacher and i am thoroughl...,Make this rhyme.,"Sure, here's the rhyme:\n\nI'm a kindergarten ..."


In [23]:
rewrite_data_df['original_text']

0        i just feel really helpless and heavy hearted
1    ive enjoyed being able to slouch about relax a...
2    i gave up my internship with the dmrg and am f...
3                           i dont know i feel so lost
4    i am a kindergarten teacher and i am thoroughl...
Name: original_text, dtype: object

In [24]:
rewrite_data_df['rewrite_prompt']

0           how to find an internship.
1           how to find an internship.
2    Explain this to me like I'm five.
3           how to find an internship.
4                     Make this rhyme.
Name: rewrite_prompt, dtype: object

In [27]:
rewrite_data_df['rewritten_text'][2]

"It's okay to feel sad when you give up an internship, even if it's the DMRG. It's like the end of a fun adventure, but it doesn't mean the world is ending.\n\nImagine you're baking a cake: You've gathered all the ingredients and started mixing, but then something goes wrong and you have to start over. It's frustrating, right? But you don't give up because you know that baking a cake takes"